# 📊 Instagram Rétrospective - Analyse Personnalisée

Analysez vos données Instagram sur n'importe quelle période.
instagram-retrospective-analyse-personnalisee

## 📋 Instructions rapides
1. Téléchargez vos données Instagram
2. Placez le dossier `your_instagram_activity` dans le même répertoire
3. **Configurez la période d'analyse** dans la cellule suivante
4. Exécutez les cellules dans l'ordre

## 1️⃣ Configuration

In [ ]:
import json
import os
from pathlib import Path
from collections import Counter
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# Style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

def fix_encoding(text):
    try:
        return text.encode('latin1').decode('utf-8')
    except:
        return text

# ⚙️ CONFIGURATION
YOUR_NAME_PATTERN = "Maxim"  # Modifiez avec votre prénom

# 📅 PÉRIODE D'ANALYSE - Choisissez l'une des options:

# Option 1: Analyser une année complète
YEAR_FILTER = 2025
START_DATE = None
END_DATE = None

# Option 2: Analyser une période spécifique (décommentez les lignes ci-dessous)
# START_DATE = datetime(2025, 1, 1)   # Début: 1er janvier 2025
# END_DATE = datetime(2025, 6, 30)    # Fin: 30 juin 2025  
# YEAR_FILTER = None  # Mettre à None quand vous utilisez START_DATE/END_DATE

# Option 3: Analyser les 6 derniers mois
# from datetime import timedelta
# END_DATE = datetime.now()
# START_DATE = END_DATE - timedelta(days=180)
# YEAR_FILTER = None

def filter_timestamp(timestamp):
    """Vérifie si un timestamp est dans la période sélectionnée"""
    if YEAR_FILTER:
        return timestamp.year == YEAR_FILTER
    elif START_DATE and END_DATE:
        return START_DATE <= timestamp <= END_DATE
    else:
        return True

# Afficher la période
if YEAR_FILTER:
    period_text = f"Année {YEAR_FILTER}"
elif START_DATE and END_DATE:
    period_text = f"Du {START_DATE.strftime('%d/%m/%Y')} au {END_DATE.strftime('%d/%m/%Y')}"
else:
    period_text = "Toutes les données"

print(f"✅ Configuration terminée")
print(f"📅 Période d'analyse: {period_text}")

## 2️⃣ Analyse des Messages

In [ ]:
base_path = Path("your_instagram_activity/messages/inbox")

stats = {
    "total_conversations": 0,
    "total_messages": 0,
    "messages_sent": 0,
    "messages_received": 0,
    "total_chars_sent": 0,
    "total_chars_received": 0,
    "conversations_data": [],
    "hourly_distribution": Counter(),
    "daily_distribution": Counter(),
    "monthly_distribution": Counter(),
}

print(f"🔄 Analyse en cours pour: {period_text}...")

for conv_folder in base_path.iterdir():
    if not conv_folder.is_dir():
        continue
    
    for json_file in conv_folder.glob("*.json"):
        with open(json_file, 'r', encoding='utf-8') as f:
            data = json.load(f)
        
        participants = [fix_encoding(p['name']) for p in data.get('participants', [])]
        your_name = next((p for p in participants if YOUR_NAME_PATTERN in p), None)
        
        if not your_name:
            continue
        
        title = fix_encoding(data.get("title", "Unknown"))
        
        conv_stats = {
            "name": title,
            "total": 0,
            "sent": 0,
            "received": 0,
            "chars_sent": 0,
            "chars_received": 0,
        }
        
        for msg in data.get("messages", []):
            content = msg.get("content")
            if not content:
                continue
            
            timestamp = datetime.fromtimestamp(msg["timestamp_ms"] / 1000)
            
            if not filter_timestamp(timestamp):
                continue
            
            content = fix_encoding(content)
            stats["total_messages"] += 1
            conv_stats["total"] += 1
            
            sender = fix_encoding(msg["sender_name"])
            
            stats["hourly_distribution"][timestamp.hour] += 1
            stats["daily_distribution"][timestamp.strftime("%A")] += 1
            stats["monthly_distribution"][timestamp.strftime("%Y-%m")] += 1
            
            if sender == your_name:
                stats["messages_sent"] += 1
                stats["total_chars_sent"] += len(content)
                conv_stats["sent"] += 1
                conv_stats["chars_sent"] += len(content)
            else:
                stats["messages_received"] += 1
                stats["total_chars_received"] += len(content)
                conv_stats["received"] += 1
                conv_stats["chars_received"] += len(content)
        
        if conv_stats["total"] > 0:
            stats["total_conversations"] += 1
            stats["conversations_data"].append(conv_stats)

stats["conversations_data"].sort(key=lambda x: x["total"], reverse=True)

print(f"\n✅ Analyse terminée!")
print(f"\n📊 STATISTIQUES - {period_text}")
print(f"Conversations: {stats['total_conversations']}")
print(f"Messages totaux: {stats['total_messages']:,}")
if stats['total_messages'] > 0:
    print(f"Envoyés: {stats['messages_sent']:,} ({stats['messages_sent']/stats['total_messages']*100:.1f}%)")
    print(f"Reçus: {stats['messages_received']:,} ({stats['messages_received']/stats['total_messages']*100:.1f}%)")
    if stats['messages_sent'] > 0:
        print(f"Moyenne chars/msg envoyé: {stats['total_chars_sent']/stats['messages_sent']:.1f}")
else:
    print("⚠️ Aucun message trouvé pour cette période")

if stats['total_messages'] > 0:
    print(f"\n🔥 TOP 10 CONVERSATIONS")
    for i, conv in enumerate(stats["conversations_data"][:10], 1):
        print(f"{i}. {conv['name']}: {conv['total']:,} msgs")

### 📈 Evolution Mensuelle - Top 5

In [ ]:
if stats['total_messages'] == 0:
    print("⚠️ Pas de données à afficher")
else:
    top_5_convs = stats["conversations_data"][:5]
    top_5_names = [c['name'] for c in top_5_convs]
    monthly_data = {name: Counter() for name in top_5_names}

    for conv_folder in base_path.iterdir():
        if not conv_folder.is_dir():
            continue
        
        for json_file in conv_folder.glob("*.json"):
            with open(json_file, 'r', encoding='utf-8') as f:
                data = json.load(f)
            
            participants = [fix_encoding(p['name']) for p in data.get('participants', [])]
            your_name = next((p for p in participants if YOUR_NAME_PATTERN in p), None)
            
            if not your_name:
                continue
            
            title = fix_encoding(data.get("title", "Unknown"))
            
            if title not in top_5_names:
                continue
            
            for msg in data.get("messages", []):
                if not msg.get("content"):
                    continue
                
                timestamp = datetime.fromtimestamp(msg["timestamp_ms"] / 1000)
                
                if not filter_timestamp(timestamp):
                    continue
                
                month = timestamp.strftime("%Y-%m")
                monthly_data[title][month] += 1

    fig, ax = plt.subplots(figsize=(16, 8))
    fig.patch.set_facecolor('#f8f9fa')
    ax.set_facecolor('white')

    all_months = sorted(set(month for person_data in monthly_data.values() for month in person_data.keys()))
    colors_palette = ['#E1306C', '#4267B2', '#34B7F1', '#FF6B6B', '#4ECDC4']

    for idx, (name, color) in enumerate(zip(top_5_names, colors_palette)):
        counts = [monthly_data[name][month] for month in all_months]
        ax.plot(all_months, counts, marker='o', linewidth=3, markersize=8, 
                label=name[:25], color=color, markerfacecolor=color,
                markeredgecolor='white', markeredgewidth=2)

    ax.set_xlabel('Mois', fontsize=14, fontweight='bold')
    ax.set_ylabel('Nombre de messages', fontsize=14, fontweight='bold')
    ax.set_title(f'Evolution Mensuelle - Top 5 Conversations ({period_text})', fontsize=18, fontweight='bold', pad=15)
    ax.legend(loc='best', fontsize=11, frameon=True, shadow=True)
    ax.grid(True, alpha=0.3, linestyle='--')
    ax.tick_params(axis='x', rotation=45, labelsize=11)
    ax.tick_params(axis='y', labelsize=11)

    for name, color in zip(top_5_names, colors_palette):
        if monthly_data[name]:
            peak_month, peak_count = monthly_data[name].most_common(1)[0]
            if peak_month in all_months:
                month_idx = all_months.index(peak_month)
                ax.annotate(f'{peak_count:,}', 
                            xy=(month_idx, peak_count),
                            xytext=(0, 10), textcoords='offset points',
                            ha='center', fontsize=9, fontweight='bold',
                            color=color,
                            bbox=dict(boxstyle='round,pad=0.3', facecolor='white', 
                                     edgecolor=color, linewidth=1.5, alpha=0.9))

    plt.tight_layout()
    plt.savefig('top5_evolution_mensuelle.png', dpi=150, bbox_inches='tight', facecolor='#f8f9fa')
    plt.show()

    print("\n✅ Graphique sauvegardé: top5_evolution_mensuelle.png")

### 📊 Graphiques Messages

In [ ]:
if stats['total_messages'] == 0:
    print("⚠️ Pas de données à afficher")
else:
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    fig.suptitle(f'Statistiques Messages Instagram - {period_text}', fontsize=20, fontweight='bold', y=0.995)
    fig.patch.set_facecolor('#f8f9fa')

    colors = {'primary': '#E1306C', 'secondary': '#4267B2', 'accent': '#34B7F1'}

    # 1. Top 10 Conversations
    ax1 = axes[0, 0]
    ax1.set_facecolor('white')
    top_10 = stats["conversations_data"][:10]
    names = [c['name'][:25] for c in top_10]
    totals = [c['total'] for c in top_10]
    bars = ax1.barh(names, totals, color=colors['primary'], alpha=0.8)
    ax1.set_xlabel('Messages', fontsize=12, fontweight='bold')
    ax1.set_title('Top 10 Conversations', fontsize=14, fontweight='bold', pad=10)
    ax1.invert_yaxis()
    ax1.grid(axis='x', alpha=0.3, linestyle='--')
    for bar, count in zip(bars, totals):
        width = bar.get_width()
        ax1.text(width + max(totals)*0.01, bar.get_y() + bar.get_height()/2, 
                 f'{count:,}', va='center', fontweight='bold', fontsize=10)

    # 2. Répartition Envoyés/Reçus
    ax2 = axes[0, 1]
    ax2.set_facecolor('white')
    pie_data = [stats['messages_sent'], stats['messages_received']]
    pie_labels = [f"Envoyés\n{stats['messages_sent']:,}", f"Reçus\n{stats['messages_received']:,}"]
    wedges, texts, autotexts = ax2.pie(pie_data, labels=pie_labels, autopct='%1.1f%%', 
                                         startangle=90, colors=[colors['secondary'], colors['accent']],
                                         textprops={'fontsize': 11, 'fontweight': 'bold'})
    for autotext in autotexts:
        autotext.set_color('white')
        autotext.set_fontsize(12)
    ax2.set_title('Répartition Messages', fontsize=14, fontweight='bold', pad=10)

    # 3. Distribution Horaire
    ax3 = axes[1, 0]
    ax3.set_facecolor('white')
    hours = sorted(stats["hourly_distribution"].keys())
    hour_counts = [stats["hourly_distribution"][h] for h in hours]
    ax3.plot(hours, hour_counts, marker='o', linewidth=3, markersize=7, 
             color=colors['primary'], markerfacecolor=colors['primary'],
             markeredgecolor='white', markeredgewidth=2)
    ax3.fill_between(hours, hour_counts, alpha=0.3, color=colors['primary'])
    ax3.set_xlabel('Heure', fontsize=12, fontweight='bold')
    ax3.set_ylabel('Messages', fontsize=12, fontweight='bold')
    ax3.set_title('Activité par Heure', fontsize=14, fontweight='bold', pad=10)
    ax3.set_xticks(range(0, 24, 2))
    ax3.grid(True, alpha=0.3, linestyle='--')
    if stats['hourly_distribution']:
        peak_hour = stats['hourly_distribution'].most_common(1)[0]
        ax3.axvline(x=peak_hour[0], color='red', linestyle='--', alpha=0.5, linewidth=2)
        ax3.text(peak_hour[0], max(hour_counts)*0.95, f'Peak: {peak_hour[0]}h', 
                 ha='center', fontsize=10, fontweight='bold', color='red',
                 bbox=dict(boxstyle='round,pad=0.5', facecolor='white', edgecolor='red', linewidth=1.5))

    # 4. Distribution par Jour
    ax4 = axes[1, 1]
    ax4.set_facecolor('white')
    days_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
    days_fr = ['Lundi', 'Mardi', 'Mercredi', 'Jeudi', 'Vendredi', 'Samedi', 'Dimanche']
    day_counts = [stats["daily_distribution"][d] for d in days_order]
    colors_days = ['#4267B2', '#5B7FC7', '#7497DB', '#8DAFF0', '#A6C7FF', '#BFD9FF', '#D8EBFF']
    bars = ax4.bar(days_fr, day_counts, color=colors_days, alpha=0.8)
    ax4.set_ylabel('Messages', fontsize=12, fontweight='bold')
    ax4.set_title('Activité par Jour', fontsize=14, fontweight='bold', pad=10)
    ax4.tick_params(axis='x', rotation=45)
    ax4.grid(axis='y', alpha=0.3, linestyle='--')
    for bar, count in zip(bars, day_counts):
        height = bar.get_height()
        if height > 0:
            ax4.text(bar.get_x() + bar.get_width()/2., height, 
                     f'{count:,}', ha='center', va='bottom', fontsize=9, fontweight='bold')

    plt.tight_layout()
    plt.savefig('messages_stats.png', dpi=150, bbox_inches='tight', facecolor='#f8f9fa')
    plt.show()

    print("\n✅ Graphique sauvegardé: messages_stats.png")

## 3️⃣ Analyse des Réactions

In [ ]:
reactions_stats = {
    "reactions_given": 0,
    "reactions_received": 0,
    "reactions_given_by_emoji": Counter(),
    "reactions_received_by_emoji": Counter(),
    "top_reactors": Counter(),
}

print(f"🔄 Analyse des réactions pour: {period_text}...")

for conv_folder in base_path.iterdir():
    if not conv_folder.is_dir():
        continue
    
    for json_file in conv_folder.glob("*.json"):
        with open(json_file, 'r', encoding='utf-8') as f:
            data = json.load(f)
        
        participants = [fix_encoding(p['name']) for p in data.get('participants', [])]
        your_name = next((p for p in participants if YOUR_NAME_PATTERN in p), None)
        
        if not your_name:
            continue
        
        for msg in data.get("messages", []):
            timestamp = datetime.fromtimestamp(msg["timestamp_ms"] / 1000)
            
            if not filter_timestamp(timestamp):
                continue
            
            reactions = msg.get("reactions", [])
            
            for reaction in reactions:
                actor = fix_encoding(reaction["actor"])
                emoji = fix_encoding(reaction["reaction"])
                
                if actor == your_name:
                    reactions_stats["reactions_given"] += 1
                    reactions_stats["reactions_given_by_emoji"][emoji] += 1
                else:
                    reactions_stats["reactions_received"] += 1
                    reactions_stats["reactions_received_by_emoji"][emoji] += 1
                    reactions_stats["top_reactors"][actor] += 1

print(f"\n✅ Analyse terminée!")
print(f"\nRéactions données: {reactions_stats['reactions_given']:,}")
print(f"Réactions reçues: {reactions_stats['reactions_received']:,}")

if reactions_stats["reactions_given_by_emoji"]:
    print(f"\nTOP 5 EMOJIS DONNÉS")
    for emoji, count in reactions_stats["reactions_given_by_emoji"].most_common(5):
        print(f"{emoji}: {count:,}")

if reactions_stats["reactions_received_by_emoji"]:
    print(f"\nTOP 5 EMOJIS REÇUS")
    for emoji, count in reactions_stats["reactions_received_by_emoji"].most_common(5):
        print(f"{emoji}: {count:,}")

### 📊 Graphiques Réactions

In [ ]:
if reactions_stats['reactions_given'] == 0 and reactions_stats['reactions_received'] == 0:
    print("⚠️ Pas de réactions pour cette période")
else:
    fig, axes = plt.subplots(1, 3, figsize=(18, 5))
    fig.suptitle(f'Statistiques Réactions Instagram - {period_text}', fontsize=20, fontweight='bold', y=1.02)
    fig.patch.set_facecolor('#f8f9fa')

    # 1. Pie chart répartition
    ax1 = axes[0]
    ax1.set_facecolor('white')
    reaction_data = [reactions_stats['reactions_given'], reactions_stats['reactions_received']]
    reaction_labels = [f"Données\n{reactions_stats['reactions_given']:,}", 
                       f"Reçues\n{reactions_stats['reactions_received']:,}"]
    wedges, texts, autotexts = ax1.pie(reaction_data, labels=reaction_labels, autopct='%1.1f%%', 
                                         startangle=90, colors=['#FF6B6B', '#4ECDC4'],
                                         textprops={'fontsize': 11, 'fontweight': 'bold'})
    for autotext in autotexts:
        autotext.set_color('white')
        autotext.set_fontsize(12)
    ax1.set_title('Répartition Réactions', fontsize=14, fontweight='bold', pad=10)

    # 2. Top Emojis Donnés
    ax2 = axes[1]
    ax2.set_facecolor('white')
    top_emojis_given = reactions_stats["reactions_given_by_emoji"].most_common(8)
    if top_emojis_given:
        emojis = [e[0] for e in top_emojis_given]
        counts = [e[1] for e in top_emojis_given]
        bars = ax2.barh(range(len(emojis)), counts, color='#FF6B6B', alpha=0.8)
        ax2.set_yticks(range(len(emojis)))
        ax2.set_yticklabels(emojis, fontsize=16)
        ax2.set_xlabel('Nombre', fontsize=12, fontweight='bold')
        ax2.set_title('Top Emojis Donnés', fontsize=14, fontweight='bold', pad=10)
        ax2.invert_yaxis()
        ax2.grid(axis='x', alpha=0.3, linestyle='--')
        for bar, count in zip(bars, counts):
            width = bar.get_width()
            ax2.text(width + max(counts)*0.01, bar.get_y() + bar.get_height()/2, 
                     f'{count:,}', va='center', fontweight='bold', fontsize=10)
    else:
        ax2.text(0.5, 0.5, 'Pas de données', ha='center', va='center', fontsize=14)
        ax2.set_xlim(0, 1)
        ax2.set_ylim(0, 1)

    # 3. Top Emojis Reçus
    ax3 = axes[2]
    ax3.set_facecolor('white')
    top_emojis_received = reactions_stats["reactions_received_by_emoji"].most_common(8)
    if top_emojis_received:
        emojis = [e[0] for e in top_emojis_received]
        counts = [e[1] for e in top_emojis_received]
        bars = ax3.barh(range(len(emojis)), counts, color='#4ECDC4', alpha=0.8)
        ax3.set_yticks(range(len(emojis)))
        ax3.set_yticklabels(emojis, fontsize=16)
        ax3.set_xlabel('Nombre', fontsize=12, fontweight='bold')
        ax3.set_title('Top Emojis Reçus', fontsize=14, fontweight='bold', pad=10)
        ax3.invert_yaxis()
        ax3.grid(axis='x', alpha=0.3, linestyle='--')
        for bar, count in zip(bars, counts):
            width = bar.get_width()
            ax3.text(width + max(counts)*0.01, bar.get_y() + bar.get_height()/2, 
                     f'{count:,}', va='center', fontweight='bold', fontsize=10)
    else:
        ax3.text(0.5, 0.5, 'Pas de données', ha='center', va='center', fontsize=14)
        ax3.set_xlim(0, 1)
        ax3.set_ylim(0, 1)

    plt.tight_layout()
    plt.savefig('reactions_stats.png', dpi=150, bbox_inches='tight', facecolor='#f8f9fa')
    plt.show()

    print("\n✅ Graphique sauvegardé: reactions_stats.png")

## 🎉 Analyse Terminée!

### 📁 Fichiers générés:
- `top5_evolution_mensuelle.png`
- `messages_stats.png`
- `reactions_stats.png`

### 💡 Pour changer de période:
Retournez dans la première cellule et modifiez les paramètres `YEAR_FILTER`, `START_DATE`, ou `END_DATE`, puis ré-exécutez toutes les cellules.